# 🏥 Adaptive Learning Assistant for Rational Antibiotic Use & AMR

## 📋 Overview
This notebook contains the complete, verified implementation of the **Basic Adaptive RAG Architecture** for Antimicrobial Stewardship. 

It is designed to strictly follow the **"Single Query Restructure"** workflow, ensuring all feedback loops (Retrieval Failure, Irrelevance) return to a single central logic node for correction.

## 🚨 Safety & Scope Disclaimer
- **Educational Only**: This system explains mechanisms and guidelines. It does NOT diagnose or treat.
- **No Prescriptions**: The model is strictly prohibited from suggesting dosages or specific treatments for a patient.
- **Source of Truth**: Answers are grounded *only* in the retrieved vector context.

---

## 🏗️ Architecture & Pipeline Flow

The code below implements the following STRICT logic flow:

1.  **START**: User Query.
2.  **🧩 NODE 1: Query Analysis & Restructuring (Central Control)**
    - *Action*: Check Relevance, Identify Category, Restructure Query, Set Tone.
    - *Decision*: If Not Relevant ➔ **STOP**.
3.  **🔢 Embedding**: Convert `Restructured Query` to Vector.
4.  **🌲 Retrieval**: Fetch Top-K Contexts from Pinecone.
5.  **⚖️ NODE 2: Retrieval Grader**
    - *Action*: Check if context supports the query.
    - *Loop*: If **BAD** ➔ **GO TO NODE 1 (Restructure Query)**.
6.  **✍️ NODE 3: Answer Generator**
    - *Action*: Generate answer using *only* context, adhering to Category & Tone.
7.  **🛡️ NODE 4: Hallucination Checker**
    - *Action*: Verify answer against context.
    - *Loop*: If **YES (Hallucinated)** ➔ **REGENERATE ANSWER (Local Loop)**.
8.  **🎯 NODE 5: Relevance Checker**
    - *Action*: Verify answer addresses Original User Query.
    - *Loop*: If **NO (Not Relevant)** ➔ **GO TO NODE 1 (Restructure Query)**.
9.  **🏁 END**: Final Verified Answer.

--- 
### 🟢 Cell 1: Library Installation
**Purpose**: Install all required Python packages for the pipeline.
- `pinecone-client`: for Vector DB connection.
- `gradio`: for the chat interface.
- `sentence-transformers`: for local query embedding.
- `requests`: for API calls to the LLM.

In [ ]:
!pip install pinecone-client gradio numpy requests sentence-transformers

### 🟢 Cell 2: LLM Connection Setup
**Purpose**: Configure the connection to the LLaMA-70B model.
We use a `call_llm` wrapper function to handle all prompt interactions consistently.

In [ ]:
import requests
import json

# 🔑 USER INPUT: API Credentials
API_KEY = "" # @param {type:"string"}
BASE_URL = "https://api.groq.com/openai/v1" # @param {type:"string"}

# 🧠 MODEL: LLaMA 70B (Strict Requirement)
MODEL_NAME = "llama-3.3-70b-versatile"

def call_llm(messages, temperature=0.3):
    """
    Sends a message list to the LLM and returns the text response.
    Handles errors gracefully.
    """
    if not API_KEY:
        return "❌ Error: API Key is missing. Please set it in Cell 2."

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": temperature
    }
    
    try:
        response = requests.post(f"{BASE_URL}/chat/completions", headers=headers, json=payload)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        print(f"❌ LLM Call Failed: {e}")
        return None

### 🟢 Cell 3: Pinecone Configuration
**Purpose**: Initialize the connection to your specific Pinecone Index.
**Assumption**: The index `PINECONE_INDEX_NAME` already exists and contains your embedded medical documents.

In [ ]:
from pinecone import Pinecone

# 🌲 PINECONE CREDENTIALS
PINECONE_API_KEY = "" # @param {type:"string"}
PINECONE_INDEX_NAME = "" # @param {type:"string"}

# Initialize Client
try:
    if PINECONE_API_KEY:
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index = pc.Index(PINECONE_INDEX_NAME)
        print(f"✅ Successfully connected to Pinecone Index: {PINECONE_INDEX_NAME}")
    else:
        print("⚠️ Warning: PINECONE_API_KEY not set. Retrieval will be simulated.")
        index = None
except Exception as e:
    print(f"❌ Pinecone Connection Error: {e}")
    index = None

### 🟢 Cell 4: Embedding Function
**Purpose**: Convert the *Restructured Query* into a vector for searching.
We use `sentence-transformers/all-MiniLM-L6-v2` locally. In a production sync, this must match your document embedding model.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load Model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def get_query_embedding(text):
    """
    Generates a vector embedding for the input text.
    """
    if not text: return []
    return embedder.encode(text).tolist()

--- 
## 🤖 Agent Logic Modules
The following cells implement the individual "Brain Nodes" of the architecture.

### 🟢 Cell 5: Query Analysis & Restructuring Node (The Central Node)
**Role**: The Orchestrator calls this first. It checks relevance and rewrites the query.
**Feedback Logic**: It accepts a `feedback_reason` (optional). If provided, it knows the previous attempt failed and re-writes the query accordingly.

In [ ]:
def agent_analyze_query(user_query, feedback_reason=None):
    """
    Analyzes the user query. 
    If feedback_reason is present, it uses that to improve the restructured query.
    """

    # 📝 SYSTEM PROMPT
    system_prompt = """
You are a medical learning query analyzer.

Tasks:
1. Determine if the query is relevant to antibiotic use or antimicrobial resistance (AMR).
2. Classify the query into ONE category:
   - Infection Context Explanation
   - Antibiotic Class Reasoning
   - Resistance Mechanism
   - Stewardship Principle
   - Safety / Adverse Effects
   - Guideline Explanation
3. Rewrite the query to optimize retrieval for a vector database.
4. Decide the appropriate answer tone (Simplified educational vs Structured clinical).

STRICT Output Format:
CATEGORY: [Category Name]
RESTRUCTURED_QUERY: [New Query]
ANSWER_TONE: [Tone]

If NOT relevant to antibiotics/AMR:
Output: NOT_RELEVANT
    """

    # 🔄 Dynamic User Prompt based on Loop State
    if feedback_reason:
        user_content = f"User Query: {user_query}\n\n⚠️ PREVIOUS FAILURE: {feedback_reason}.\nACTION: You MUST rewrite the query differently to address this failure."
    else:
        user_content = f"User Query: {user_query}"

    # Call LLM
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
    response = call_llm(messages)

    # 🧠 Parsing Logic
    result = {
        "is_relevant": True,
        "category": "General",
        "restructured_query": user_query,
        "tone": "Educational"
    }

    if "NOT_RELEVANT" in response:
        result["is_relevant": False]
        return result

    # Simple parsing of the strict keys
    lines = response.split('\n')
    for line in lines:
        if line.startswith("CATEGORY:"):
            result["category"] = line.replace("CATEGORY:", "").strip()
        elif line.startswith("RESTRUCTURED_QUERY:"):
            result["restructured_query"] = line.replace("RESTRUCTURED_QUERY:", "").strip()
        elif line.startswith("ANSWER_TONE:"):
            result["tone"] = line.replace("ANSWER_TONE:", "").strip()

    return result

### 🟢 Cell 6: Retrieval Grader Node
**Role**: Evaluates if the Context retrieved from Pinecone is actually useful.
**Logic**: Returns `GOOD` or `BAD`. If BAD, the Orchestrator will trigger a loop.

In [ ]:
def agent_grade_retrieval(query, contexts):
    context_str = "\n".join(contexts)
    
    prompt = f"""
    User Query: {query}
    Retrieved Context: {context_str}
    
    Task: Is the context relevant and sufficient to answer the query?
    Output: Answer ONLY 'GOOD' or 'BAD'.
    """
    
    response = call_llm([{"role": "user", "content": prompt}])
    # Safety fallback
    if not response: return "BAD"
    
    return "BAD" if "BAD" in response.upper() else "GOOD"

### 🟢 Cell 7: Answer Generator Node
**Role**: Synthesizes the final answer using *only* the context.

In [ ]:
def agent_generate_answer(query, contexts, category, tone):
    context_str = "\n".join(contexts)
    
    system_prompt = f"""
You are an educational medical assistant.
Use ONLY the provided context.
Follow the answer tone: {tone} and category: {category} strictly.

Category-specific guidance:
- Infection context → general principles
- Antibiotic class → spectrum & resistance risks
- Resistance mechanism → biological explanation
- Stewardship → safety & AMR prevention
- Safety → adverse effects & caution
- Guidelines → rationale, not instructions

Constraints:
- No prescribing
- No recommendations
- Use cautious language
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Context: {context_str}\n\nQuestion: {query}"}
    ]
    
    return call_llm(messages)

### 🟢 Cell 8: Hallucination Checker Node
**Role**: Ensures safety. Checks if the answer contains claims *not* in the source text.
**Logic**: Returns `YES` (it is hallucinated) or `NO` (it is safe).

In [ ]:
def agent_check_hallucination(answer, contexts):
    context_str = "\n".join(contexts)
    prompt = f"""
    Context: {context_str}
    Generated Answer: {answer}
    
    Does the answer contain any claim NOT supported by the retrieved context?
    Output: Answer ONLY 'YES' or 'NO'.
    """
    response = call_llm([{"role": "user", "content": prompt}])
    if not response: return "NO" # Assume safe if check fails to prevent blocking
    return "YES" if "YES" in response.upper() else "NO"

### 🟢 Cell 9: Relevance Checker Node
**Role**: Ensures the answer actually helps the user.
**Logic**: Returns `YES` (Relevant) or `NO` (Not Relevant).

In [ ]:
def agent_check_relevance(answer, original_query):
    prompt = f"""
    Original Query: {original_query}
    Generated Answer: {answer}
    
    Does the answer fully and clearly address the original query regarding AMR?
    Output: Answer ONLY 'YES' or 'NO'.
    """
    response = call_llm([{"role": "user", "content": prompt}])
    if not response: return "YES"
    return "YES" if "YES" in response.upper() else "NO"

--- 
## ⚙️ Orchestrator (The Main Pipeline)

### 🟢 Cell 10: The Workflow Control Loop
This function ties everything together. It implements the **While Loop** that allows the system to self-correct.

**Feedback Logic map:**
1. `Retrieval == BAD` ➔ Loop back to `agent_analyze_query` (Step 1).
2. `Hallucination == YES` ➔ Loop back to `agent_generate_answer` (Step 5) *locally*.
3. `Relevance == NO` ➔ Loop back to `agent_analyze_query` (Step 1).

In [ ]:
def adaptive_rag_orchestrator(user_query):
    MAX_RETRIES = 3
    attempt = 0
    logs = []
    
    # Holds feedback for the next loop iteration (if needed)
    feedback_reason = None
    
    while attempt < MAX_RETRIES:
        attempt += 1
        logs.append(f"\n--- 🔄 Cycle {attempt} Start ---")
        
        # --- STEP 1: ANALYSIS ---
        analysis = agent_analyze_query(user_query, feedback_reason)
        
        if not analysis["is_relevant"]:
            logs.append("🛑 Stopped: Query identified as Not Relevant.")
            return "I can only answer questions related to Antimicrobial Stewardship and AMR.", logs
            
        restructured_q = analysis["restructured_query"]
        category = analysis["category"]
        tone = analysis["tone"]
        logs.append(f"🔍 Analyzed: Category=[{category}] | New Query=[{restructured_q}]")
        
        # --- STEP 2: EMBEDDING ---
        vector = get_query_embedding(restructured_q)
        
        # --- STEP 3: RETRIEVAL ---
        if index:
            results = index.query(vector=vector, top_k=3, include_metadata=True)
            matches = results.get('matches', [])
            contexts = [m['metadata']['text'] for m in matches if 'text' in m['metadata']]
        else:
            # Mock for demo if no DB connected
            contexts = ["[MOCK CONTEXT] Bacteria develop resistance through mutation... Stewardship requires right drug, right dose."]

        # --- STEP 4: RETRIEVAL GRADING ---
        grade = agent_grade_retrieval(restructured_q, contexts)
        if grade == "BAD":
            logs.append("⚠️ Retrieval Grader: BAD -> Looping back to Restructure.")
            feedback_reason = "Previous query yielded poor/irrelevant documents"
            continue # ↩️ LOOP TO STEP 1
        else:
            logs.append("✅ Retrieval Grader: GOOD")
            
        # --- STEP 5: GENERATION ---
        answer = agent_generate_answer(restructured_q, contexts, category, tone)
        
        # --- STEP 6: VALIDATION GATES ---
        
        # A. Hallucination Check
        is_hallucinated = agent_check_hallucination(answer, contexts)
        if is_hallucinated == "YES":
            logs.append("⚠️ Hallucination Detected -> Regenerating (Local correction).")
            # Local retry (one shot)
            answer = agent_generate_answer(restructured_q, contexts, category, tone)
        
        # B. Relevance Check
        is_relevant_answer = agent_check_relevance(answer, user_query)
        if is_relevant_answer == "NO":
            logs.append("⚠️ Answer Relevance: NO -> Looping back to Restructure.")
            feedback_reason = "Generated answer did not fully address the user intent"
            continue # ↩️ LOOP TO STEP 1
            
        # ✅ SUCCESS
        logs.append("✅ Verification Passed: Sending Final Answer.")
        final_output = f"**Category:** {category}\n\n{answer}"
        return final_output, logs

    return "❌ Sorry, I tried multiple times but couldn't generate a verified answer. Please try rephrasing.", logs

### 🟢 Cell 11: Main Interface (Gradio)
**Purpose**: Launch the user interface.
Run this cell to generate the public or local link.

In [ ]:
import gradio as gr

def ui_handler(query):
    response, logs = adaptive_rag_orchestrator(query)
    return response, "\n".join(logs)

with gr.Blocks(title="Adaptive AMR Assistant") as demo:
    gr.Markdown("# 🛡️ Adaptive AMR Stewardship Assistant")
    gr.Markdown("**Educational Tool** | Powered by Adaptive RAG & LLaMA-70B")
    
    with gr.Row():
        query_input = gr.Textbox(label="Enter your question about Antibiotics/AMR", placeholder="e.g., Why avoid Ciprofloxacin in simple UTI?")
        submit_btn = gr.Button("Analyze & Search", variant="primary")
    
    with gr.Row():
        answer_output = gr.Markdown(label="Verified Answer")
        log_output = gr.Textbox(label="Pipeline Application Logs", lines=10)
        
    submit_btn.click(fn=ui_handler, inputs=query_input, outputs=[answer_output, log_output])
    
demo.launch()